In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from concurrent import futures
import requests
from bs4 import BeautifulSoup
import re

In [90]:
df = pd.read_csv('main_task.csv')
df.loc[df['City'] == 'Oporto', 'City'] = 'Porto'
cities = df.City.unique()

In [91]:
df = df.loc[~df.URL_TA.duplicated(keep='first')]

In [92]:
df.head(1)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643


In [140]:
URL = df['URL_TA']
HEADERS = {'user-agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.152 YaBrowser/21.2.2.101 Yowser/2.5 Safari/537.36',
           'accept': '*/*'}


def get_html(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params)
    return r


def get_coord(url):
    r = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(r.text, 'html.parser')
    coord_finds = soup.find(content=re.compile(
        'https://maps.google.com/maps/api/'))
    if coord_finds:
        coord_finds = coord_finds.get('content').split(
            '&zoom')[0].split('?center=')[1].split('%2C')
    else:
        coord_finds = [[], []]
    return coord_finds


def get_content(html, url):
    soup = BeautifulSoup(html, 'html.parser')
    pattern = re.compile('\d+')
    meals_find = soup.find('div', string='Meals')
    adress_find = soup.find(href="#MAPVIEW")
    rub_find = soup.find('div', string=re.compile('RUB\s'))
    if meals_find:
        meals_find = meals_find.find_next('div').get_text().split(', ')
    else:
        meals_find = 'no_info'

    if adress_find:
        adress_find = adress_find.get_text()
    else:
        adress_find = 'no_info'

    if rub_find:
        rub_find = pattern.findall(rub_find.get_text().replace(',', ''))
        rub_max = float(rub_find[1])
        rub_min = float(rub_find[0])
    else:
        rub_find = 'no_info'
        rub_max = 'no_info'
        rub_min = 'no_info'
    rest_pars.append({'ID_TA': df['ID_TA'][df['URL_TA'] == url].item(),
                      'meals': meals_find,
                      'rest_adress': adress_find,
                      'rest_coord': get_coord('https://google.com/maps/search/'+adress_find),
                      'rub_max': rub_max,
                      'rub_min': rub_min
                      })


def parse(url):
    html = get_html('https://www.tripadvisor.com' + url)
    if html.status_code == 200:
        get_content(html.text, url)
    else:
        pass

In [146]:
rest_pars = []
count = 0
col = df['URL_TA'][:]

with ThreadPoolExecutor() as executor:
    for i in col:
        executor.submit(parse, i)

In [2]:
def send_telegram(text: str):
    token = "1509182364:AAF-1pY7y0cNg3tFncBcPTK0gpL_vhbJYyE"
    url = "https://api.telegram.org/bot"
    channel_id = "-1001443232386"
    method = url + token + "/sendMessage"

    r = requests.get(method, data={
         'chat_id': channel_id,
         'text': text
    })
    if r.status_code != 200:
        raise Exception("post_text error")

if __name__ == '__main__':
    send_telegram('Всё готово!')

In [150]:
data_rest = pd.DataFrame(rest_pars)
data_rest.to_csv('rest_.csv', index = False)
data_rest

,ID_TA,meals,rest_adress,rest_coord,rub_max,rub_min
0,d5808504,"[Lunch, Dinner]","Via Casella Gianbattista 10, 20100 Milan Italy","[45.49584, 9.14616]",no_info,no_info
1,d12845029,no_info,"Johannesgasse 4 im Hof, Vienna 1010 Austria","[48.2049288, 16.3719383]",no_info,no_info
2,d7992032,no_info,"39 Aarstavaegen, Stockholm 120 52 Sweden","[59.2960467, 18.0529969]",no_info,no_info
3,d6633059,"[Breakfast, Brunch, Lunch, Dinner]","6 Marino Mart DUBLIN3, Dublin Ireland","[53.3640658, -6.2328068]",no_info,no_info
4,d8632781,"[Lunch, Dinner, Late Night, Drinks]","30 North Audley Street Mayfair, London W1K 6ZF...","[51.5127973, -0.1536371]",11220,3366
...,...,...,...,...,...,...
39969,d1100838,no_info,"Ulica Miodowa 14, Warsaw 00-246 Poland","[52.2474663, 21.0094532]",2293,153
39970,d947615,no_info,"13 Stoertloppsvaegen, Stockholm 129 47 Sweden","[59.2923599, 17.9679814]",no_info,no_info
39971,d2104414,no_info,"Via Marco Formentini 7 Brera District, 20121 M...","[45.4718473, 9.186035]",6130,1751
39972,d1800036,"[Breakfast, Lunch, Dinner, Brunch, Late Night,...","4 rue de Rivoli 75004, 75004 Paris France","[48.8552932, 2.361394]",no_info,no_info


In [151]:
df.merge(data_rest, how = 'outer')

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,meals,rest_adress,rest_coord,rub_max,rub_min
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,"[Lunch, Dinner, Brunch, Breakfast, Late Night]",17 boulevard Saint Jacques Paris Marriott Rive...,"[48.8316299, 2.3397946]",3503,1314
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,no_info,"39 Aarstavaegen, Stockholm 120 52 Sweden","[59.2960467, 18.0529969]",no_info,no_info
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,"[Lunch, Dinner, Late Night, Drinks]","30 North Audley Street Mayfair, London W1K 6ZF...","[51.5127973, -0.1536371]",11220,3366
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,no_info,"Berliner Str. 54, 13189 Berlin Germany","[52.5577069, 13.4137119]",no_info,no_info
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,"[Dinner, Lunch]","Zielstattstr. 6, 81379 Munich, Bavaria Germany","[48.1039891, 11.5344284]",no_info,no_info
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39975,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,no_info,"Via Marco Formentini 7 Brera District, 20121 M...","[45.4718473, 9.186035]",6130,1751
39976,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,"[Breakfast, Lunch, Dinner, Brunch, Late Night,...","4 rue de Rivoli 75004, 75004 Paris France","[48.8552932, 2.361394]",no_info,no_info
39977,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,no_info,"13 Stoertloppsvaegen, Stockholm 129 47 Sweden","[59.2923599, 17.9679814]",no_info,no_info
39978,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,no_info,"Ulica Miodowa 14, Warsaw 00-246 Poland","[52.2474663, 21.0094532]",2293,153


In [81]:
print('https://www.tripadvisor.com' + URL[0])

https://www.tripadvisor.com/Restaurant_Review-g187147-d1912643-Reviews-R_Yves-Paris_Ile_de_France.html
